In [1]:
import json

from datasets import load_dataset
import os
import pathlib
import re
from subprocess import CalledProcessError

from utils import run_command

In [2]:
dataset = load_dataset("ASSERT-KTH/DISL", "decomposed")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
dataset

In [ ]:
contracts = {}

for row in dataset["train"]:
    v = row["compiler_version"]
    if row["language"] == "Solidity" and v.startswith("v0.8") and int(v.split('.')[2].split('+')[0]) >= 10:
        row["parsed_version"] = v.split('+')[0]
        if row["contract_address"] in contracts:
            contracts[row["contract_address"]].append(row)
        else:
            contracts[row["contract_address"]] = [row]

In [ ]:
print(len(contracts.keys()))
last_source = None


In [ ]:
def extract_yul_from_single_sol(contract) -> bool:
    run_command('rm -rf ./cache/*')
    solc = f"./compilers/solc-{contract['parsed_version']}"
    sol_orig_path = contract['file_path'].replace(' ', '_')
    node_modules = pathlib.Path().absolute() / "node_modules"
    base_path = pathlib.Path().absolute() / "cache"
    sol_path = base_path / (sol_orig_path[1:] if sol_orig_path.startswith("/") else sol_orig_path)
    yul_dir = pathlib.Path().absolute().parent / "dataset" / contract["contract_address"]
    if os.path.exists(yul_dir) and "meta.info" in os.listdir(yul_dir):
        print("Already exists")
        return True
    if os.path.exists(solc):
        # [print(x["file_path"]) for x in contract]
        print(f'Contract name: {contract["contract_name"]}')
        print(contract["compiler_version"])
        print("Sol orig path", sol_orig_path)
        print("Sol final path", sol_path)
        print("Yul dir", yul_dir)
        os.makedirs(sol_path.parent, exist_ok=True)
        with open(sol_path, "w") as f:
            f.write(contract["source_code"])
        out = run_command(f"{solc} --include-path {node_modules} --base-path {base_path}  --ir {sol_path}")
        # split outputs by IR: - separate yul sources
        for yul_source in out.split("IR:"):
            # extract name for yul file based on top Object name
            filename = None
            match = re.search(r'object "(.*)"', yul_source)
            if match:
                filename = match.groups()[0]
            
            if not filename:
                continue
                 
            # save yul into disk
            yul_path = yul_dir / f"{filename}.yul"
            os.makedirs(yul_dir, exist_ok=True)
            with open(yul_path, "w") as f:
                f.write(yul_source)
            
            # verify it compiles successfully
            run_command(f"solc --strict-assembly {yul_path}")
        
        # save metadata to have them and confirm directory is completed
        with (open(yul_dir / "meta.info", "w") as f):
            _contract = contract.copy()
            _contract.pop("source_code")
            json.dump(_contract, f)
        
        return True
    else:
        print(f"Solc {contract['parsed_version']} not found")
    
    return False

In [ ]:
nr_of_multifile = 0
for contract in contracts.values():
    # TEMP skip if more than 1 file
    #print(len(contract))
    if len(contract) > 1:
        # for x in contract:
        #     print(x['contract_name'], x['file_path'], x['parsed_version'])
        # break
        nr_of_multifile += 1
        continue
        
    try:
        # extract_yul_from_single_sol(contract[0])
        if contract[0]['contract_name'] == 'STAR':
            print(contract[0]['contract_name'])
            print(contract[0]['parsed_version'])
            print(contract[0]['file_path'])
            #print(contract[0]['source_code'])
            print(contract[0]["contract_address"])
            extract_yul_from_single_sol(contract[0])
        else:
            pass
    except CalledProcessError:
        print(f"Solc error with contract {contract[0]['contract_name']} -> {contract[0]['contract_address']}")
    except Exception as e:
        print(f"Error with contract {contract[0]['contract_name']}: {e}")
    # if extract_yul_from_single_sol(contract[0]):
    #     break

In [ ]:
nr_of_multifile